In [1]:
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import seaborn as sns
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from gensim.models import Word2Vec, KeyedVectors

from sklearn.metrics import adjusted_rand_score
import nltk
from nltk.corpus import wordnet, wordnet2021
import gensim

import multiprocessing
from gensim.models.phrases import Phrases, Phraser

from cluster_method import *

# Test Word-net

In [6]:
# Get the n-gram words
n_gram = ["natural language processing", "machine learning"]

# Get the synsets for the n-gram words
n_gram_synsets = []
for word in n_gram:
    synsets = wordnet.synsets(word)
    n_gram_synsets.append(synsets)

print(n_gram_synsets)

# Calculate the similarity between the synsets using the Wu-Palmer similarity measure
similarity_scores = []
for synset1 in n_gram_synsets[0]:
    for synset2 in n_gram_synsets[1]:
        similarity = wordnet.wup_similarity(synset1, synset2)
        similarity_scores.append(similarity)

# # Get the average similarity score
# average_similarity = sum(similarity_scores) / len(similarity_scores)

# # Print the average similarity score
# print(average_similarity)

[[], []]


In [7]:
dog_synsets = wordnet2021.synsets("Spanish")
cat_synsets = wordnet2021.synsets("cat")

# Calculate the Wu-Palmer similarity between the two synsets
wu_palmer_similarity = wordnet.wup_similarity(dog_synsets[0], cat_synsets[0])

In [8]:
wu_palmer_similarity

0.09090909090909091

# Read terms and sentences from files

In [4]:
# docs and terms from files, complete
dir = "small_teddy.txt"
text_file = open(dir, "r")
txt = text_file.read()
text_file.close()

terms = open('small_teddy_terms.txt', 'r').read().split('\n')
# terms = [term.lower() for term in terms]

In [5]:
reverse_sorted_terms = sorted(terms, reverse=True)
reverse_sorted_terms

['young Theodore',
 'years at Christ Church',
 'world tour',
 'war hero',
 'war against Spain',
 'vice president under President William',
 'vice president',
 'vast range',
 'various areas',
 'unorthodox career choice',
 'uneven coverage',
 'tropical disease',
 'tours of Europe',
 'top left',
 'sudden nighttime asthma attacks',
 'sudden death',
 'successful naval war',
 'successful efforts',
 'strenuous lifestyle',
 'state legislature',
 'state assemblyman',
 'son of businessman Cornelius Van',
 'socialite Martha Stewart Bulloch',
 'significant benefits',
 'sickly child',
 'section of Cambridge',
 'second-floor window',
 'second wife',
 'same night',
 'rudiments of taxidermy',
 'robust masculinity',
 'rhetoric courses',
 'regulation of railroads',
 'regime of exercise',
 'reform faction of Republicans',
 'reform faction',
 'range of interests',
 'pure food',
 'prominent member',
 'prominent leader',
 'progressive movement',
 'procession of Abraham Lincoln',
 'prestigious Porcellian Clu

In [6]:
new_term = []

for term in reverse_sorted_terms:
    new_term.append('_'.join(term.split()))

new_term

['young_Theodore',
 'years_at_Christ_Church',
 'world_tour',
 'war_hero',
 'war_against_Spain',
 'vice_president_under_President_William',
 'vice_president',
 'vast_range',
 'various_areas',
 'unorthodox_career_choice',
 'uneven_coverage',
 'tropical_disease',
 'tours_of_Europe',
 'top_left',
 'sudden_nighttime_asthma_attacks',
 'sudden_death',
 'successful_naval_war',
 'successful_efforts',
 'strenuous_lifestyle',
 'state_legislature',
 'state_assemblyman',
 'son_of_businessman_Cornelius_Van',
 'socialite_Martha_Stewart_Bulloch',
 'significant_benefits',
 'sickly_child',
 'section_of_Cambridge',
 'second-floor_window',
 'second_wife',
 'same_night',
 'rudiments_of_taxidermy',
 'robust_masculinity',
 'rhetoric_courses',
 'regulation_of_railroads',
 'regime_of_exercise',
 'reform_faction_of_Republicans',
 'reform_faction',
 'range_of_interests',
 'pure_food',
 'prominent_member',
 'prominent_leader',
 'progressive_movement',
 'procession_of_Abraham_Lincoln',
 'prestigious_Porcellian_Clu

In [7]:
normal_sentence = nltk.sent_tokenize(txt)

In [8]:
for i in range(len(terms)):
    txt = txt.replace(reverse_sorted_terms[i], new_term[i])
    

In [9]:
print(txt)

Theodore_Roosevelt_Jr., often referred to as Teddy or by his initials, T. R., was an American_politician, statesman, soldier, conservationist, naturalist, historian, and writer who served as the 26th_president of the United_States from 1901 to 1909. He previously served as the 25th vice_president_under_President_William McKinley from March to September 1901 and as the 33rd_governor_of_New_York from 1899 to 1900. Assuming the presidency_after_McKinley's assassination, Roosevelt emerged as a leader of the Republican_Party and became a driving force for anti-trust and Progressive_policies.

A sickly_child with debilitating_asthma, he overcame his health_problems as he grew by embracing a strenuous_lifestyle. Roosevelt integrated his exuberant_personality and a vast_range_of_interests and achievements into a "cowboy" persona defined by robust_masculinity. He was home-schooled and began a lifelong naturalist_avocation before attending Harvard_College. His book The Naval_War of 1812 (1882) e

In [10]:
new_term_sentences = nltk.sent_tokenize(txt)
# sentences = [sentence.lower() for sentence in sentences]


# vocab = {term: idx for idx, term in enumerate(terms)}

# vectorizer = TfidfVectorizer(vocabulary = terms, ngram_range=(2, 5))
# X = vectorizer.fit_transform(sentences)
# X.toarray()

In [12]:
normal_sentence

['Theodore Roosevelt Jr., often referred to as Teddy or by his initials, T. R., was an American politician, statesman, soldier, conservationist, naturalist, historian, and writer who served as the 26th president of the United States from 1901 to 1909.',
 'He previously served as the 25th vice president under President William McKinley from March to September 1901 and as the 33rd governor of New York from 1899 to 1900.',
 "Assuming the presidency after McKinley's assassination, Roosevelt emerged as a leader of the Republican Party and became a driving force for anti-trust and Progressive policies.",
 'A sickly child with debilitating asthma, he overcame his health problems as he grew by embracing a strenuous lifestyle.',
 'Roosevelt integrated his exuberant personality and a vast range of interests and achievements into a "cowboy" persona defined by robust masculinity.',
 'He was home-schooled and began a lifelong naturalist avocation before attending Harvard College.',
 'His book The N

# Test Word2vec

## Pretrained model

In [22]:
word_limit = 100000
pretrained_model_dir = "GoogleNews-vectors-negative300.bin"
# Load the Word2Vec model
model = KeyedVectors.load_word2vec_format(pretrained_model_dir, binary=True)

10/27/2023 20:12:04 - INFO - 	 loading projection weights from GoogleNews-vectors-negative300.bin
10/27/2023 20:12:23 - INFO - 	 KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from GoogleNews-vectors-negative300.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2023-10-27T20:12:23.282652', 'gensim': '4.3.0', 'python': '3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:24:38) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'load_word2vec_format'}


In [23]:
model.most_similar('Theodore')

[('Robert', 0.5458441376686096),
 ('Benjamin', 0.5424085259437561),
 ('Gerald', 0.538208544254303),
 ('Kenneth', 0.5353478789329529),
 ('D._Zucconi_Ph.D.', 0.5341265797615051),
 ('F._di_Stefano', 0.5329923629760742),
 ('Arthur', 0.5312926769256592),
 ('Ernest', 0.5262468457221985),
 ('Ronald', 0.5219574570655823),
 ('Alfred', 0.5113332271575928)]

In [24]:
# Find the analogy between the pairs of words "king" and "man" and "queen" and "woman"
analogy = model.most_similar(positive=['king', 'woman'], negative=['man'])

# Print the analogy
print(f"{analogy[0][0]}: {analogy[0][1]}")

queen: 0.7118193507194519


## Test Train

In [45]:
new_term_sentences

['Theodore_Roosevelt_Jr., often referred to as Teddy or by his initials, T. R., was an American_politician, statesman, soldier, conservationist, naturalist, historian, and writer who served as the 26th_president of the United_States from 1901 to 1909.',
 'He previously served as the 25th vice_president_under_President_William McKinley from March to September 1901 and as the 33rd_governor_of_New_York from 1899 to 1900.',
 "Assuming the presidency_after_McKinley's assassination, Roosevelt emerged as a leader of the Republican_Party and became a driving force for anti-trust and Progressive_policies.",
 'A sickly_child with debilitating_asthma, he overcame his health_problems as he grew by embracing a strenuous_lifestyle.',
 'Roosevelt integrated his exuberant_personality and a vast_range_of_interests and achievements into a "cowboy" persona defined by robust_masculinity.',
 'He was home-schooled and began a lifelong naturalist_avocation before attending Harvard_College.',
 'His book The N

In [48]:
vocab = [sentence.split() for sentence in new_term_sentences]

In [65]:
print('leader,' in vocab)

False


In [63]:
vocab

[['Theodore_Roosevelt_Jr.,',
  'often',
  'referred',
  'to',
  'as',
  'Teddy',
  'or',
  'by',
  'his',
  'initials,',
  'T.',
  'R.,',
  'was',
  'an',
  'American_politician,',
  'statesman,',
  'soldier,',
  'conservationist,',
  'naturalist,',
  'historian,',
  'and',
  'writer',
  'who',
  'served',
  'as',
  'the',
  '26th_president',
  'of',
  'the',
  'United_States',
  'from',
  '1901',
  'to',
  '1909.'],
 ['He',
  'previously',
  'served',
  'as',
  'the',
  '25th',
  'vice_president_under_President_William',
  'McKinley',
  'from',
  'March',
  'to',
  'September',
  '1901',
  'and',
  'as',
  'the',
  '33rd_governor_of_New_York',
  'from',
  '1899',
  'to',
  '1900.'],
 ['Assuming',
  'the',
  "presidency_after_McKinley's",
  'assassination,',
  'Roosevelt',
  'emerged',
  'as',
  'a',
  'leader',
  'of',
  'the',
  'Republican_Party',
  'and',
  'became',
  'a',
  'driving',
  'force',
  'for',
  'anti-trust',
  'and',
  'Progressive_policies.'],
 ['A',
  'sickly_child'

In [46]:
with open("paragraph.txt", 'w') as file:
    file.write(txt)
    file.close()

In [55]:
modelW2V = Word2Vec(min_count=1, epochs=5)

11/03/2023 16:11:41 - INFO - 	 Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=100, alpha=0.025>', 'datetime': '2023-11-03T16:11:41.353025', 'gensim': '4.3.0', 'python': '3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:24:38) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}


In [56]:
modelW2V.build_vocab(vocab, progress_per=5000)

11/03/2023 16:11:43 - INFO - 	 collecting all words and their counts
11/03/2023 16:11:43 - INFO - 	 PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
11/03/2023 16:11:43 - INFO - 	 collected 725 word types from a corpus of 1371 raw words and 87 sentences
11/03/2023 16:11:43 - INFO - 	 Creating a fresh vocabulary
11/03/2023 16:11:43 - INFO - 	 Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 725 unique words (100.00% of original 725, drops 0)', 'datetime': '2023-11-03T16:11:43.995006', 'gensim': '4.3.0', 'python': '3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:24:38) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'prepare_vocab'}
11/03/2023 16:11:43 - INFO - 	 Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 1371 word corpus (100.00% of original 1371, drops 0)', 'datetime': '2023-11-03T16:11:43.996998', 'gensim': '4.3.0', 'python': '3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:

In [57]:
modelW2V.train(vocab, total_examples=modelW2V.corpus_count, epochs=30, report_delay=1)

11/03/2023 16:12:22 - INFO - 	 Word2Vec lifecycle event {'msg': 'training model with 3 workers on 725 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5 shrink_windows=True', 'datetime': '2023-11-03T16:12:22.659381', 'gensim': '4.3.0', 'python': '3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:24:38) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'train'}
11/03/2023 16:12:22 - INFO - 	 EPOCH 0: training on 1371 raw words (964 effective words) took 0.0s, 171524 effective words/s
11/03/2023 16:12:22 - INFO - 	 EPOCH 1: training on 1371 raw words (974 effective words) took 0.0s, 1590204 effective words/s
11/03/2023 16:12:22 - INFO - 	 EPOCH 2: training on 1371 raw words (974 effective words) took 0.0s, 846957 effective words/s
11/03/2023 16:12:22 - INFO - 	 EPOCH 3: training on 1371 raw words (982 effective words) took 0.0s, 851248 effective words/s
11/03/2023 16:12:22 - INFO - 	 EPOCH 4: training on 1371 raw words 

(29000, 41130)

In [68]:
modelW2V.wv.most_similar("Theodore_Roosevelt")

[('but', 0.9556963443756104),
 ('where', 0.9547401070594788),
 ('at', 0.9544247388839722),
 ('had', 0.954113245010376),
 ('in', 0.9539525508880615),
 ('him', 0.9539323449134827),
 ('and', 0.9539197087287903),
 ('to', 0.95358806848526),
 ('he', 0.9535692930221558),
 ('the', 0.9535645842552185)]

# Coreference Resolution

In [2]:
from transformers import AutoTokenizer, AutoModel
from transformers import pipeline
import torch

d:\anaconda3\envs\StockLSTM\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from fastcoref import FCoref, LingMessCoref

In [ ]:
FCorefmodel = FCoref(device='cuda:0')
LMCorefmodel = LingMessCoref(device='cuda:0')

10/27/2023 20:10:50 - INFO - 	 missing_keys: []
10/27/2023 20:10:50 - INFO - 	 unexpected_keys: []
10/27/2023 20:10:50 - INFO - 	 mismatched_keys: []
10/27/2023 20:10:50 - INFO - 	 error_msgs: []
10/27/2023 20:10:50 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M
10/27/2023 20:11:08 - INFO - 	 missing_keys: []
10/27/2023 20:11:08 - INFO - 	 unexpected_keys: []
10/27/2023 20:11:08 - INFO - 	 mismatched_keys: []
10/27/2023 20:11:08 - INFO - 	 error_msgs: []
10/27/2023 20:11:09 - INFO - 	 Model Parameters: 590.0M, Transformer: 434.6M, Coref head: 155.4M


In [35]:
text = ' '.join(normal_sentence)
print(text)

In [ ]:
word_index_dict = {}

In [37]:
pred1 = FCorefmodel.predict(
   texts=text
)

pred2 = LMCorefmodel.predict(
    texts=text
)

10/27/2023 20:13:27 - INFO - 	 Tokenize 1 inputs...
10/27/2023 20:13:28 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|██████████| 1/1 [00:03<00:00,  3.73s/it]
10/27/2023 20:13:31 - INFO - 	 Tokenize 1 inputs...
10/27/2023 20:13:32 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|██████████| 1/1 [00:02<00:00,  2.88s/it]


In [38]:
FCoref_terms_stringtype = pred1.get_clusters(as_strings=True)
LMCorefmodel_terms_stringtype = pred2.get_clusters(as_strings=True)

FCoref_terms_indextype = pred1.get_clusters(as_strings=False)
LMCorefmodel_terms_indextype = pred2.get_clusters(as_strings=False)


In [40]:
for i in range(len(FCoref_terms_indextype)):
    print(FCoref_terms_indextype[i])

[(0, 77), (57, 60), (250, 252), (472, 481), (635, 637), (647, 650), (670, 672), (714, 723), (735, 738), (863, 865), (960, 963), (1010, 1013), (1092, 1101), (1190, 1193), (1252, 1255), (1273, 1275), (1343, 1352), (1492, 1494), (1630, 1639), (1727, 1730), (1781, 1784), (1827, 1836), (1972, 1981), (2130, 2132), (2144, 2147), (2296, 2305), (2449, 2451), (2486, 2488), (2529, 2538), (2637, 2640), (2708, 2711), (2747, 2750), (2797, 2806), (2941, 2944), (2957, 2959), (2968, 2971), (3014, 3017), (3053, 3062), (3177, 3179), (3239, 3241), (3383, 3392), (3447, 3449), (3503, 3505), (3621, 3630), (3683, 3686), (3723, 3725), (3790, 3793), (3871, 3889), (3900, 3922), (4005, 4007), (4139, 4141), (4297, 4307), (4696, 4706), (4945, 4954), (4996, 5007), (5036, 5039), (5077, 5079), (5212, 5220), (5225, 5228), (5273, 5275), (5321, 5324), (5378, 5380), (5449, 5458), (5585, 5587), (5595, 5598), (5634, 5636), (5655, 5657), (5730, 5732), (5742, 5745), (5823, 5834), (5867, 5870), (5872, 5875), (6141, 6150), (615

In [ ]:
coref_terms_clusters = []
for i in range(len(FCoref_terms_stringtype)):
    coref_terms_clusters.append(list(set(FCoref_terms_stringtype[i])))

print(coref_terms_clusters)
# print(a_false)

[['I', 'My', "Theodore's", 'him', 'the younger Roosevelt', 'Theodore Roosevelt', 'he', "Roosevelt's", 'Theodore', 'young Theodore', 'Theodore Roosevelt Jr.', 'Roosevelt', 'Theodore Roosevelt Jr., often referred to as Teddy or by his initials, T. R.,', 'His', 'his', 'your', 'He'], ['President McKinley', 'President William McKinley', 'McKinley', "McKinley's", 'his'], ['New York', 'New York City', "New York's"], ['1898'], ['the presidency'], ['the United States', 'the country', "the nation's"], ['It', 'his "Square Deal" domestic policies'], ['the Navy'], ['the 1906 Nobel Peace Prize', 'a Nobel Prize'], ['his close ally William Howard Taft', "Taft's"], ['1912'], ['the 1912 presidential election', 'the election'], ['win', 'the defeat'], ['the Democratic nominee Woodrow Wilson', 'his', 'Wilson'], ['the war', 'World War I'], ['a younger brother (Elliott)', 'Elliott', 'His'], ['His paternal grandfather', 'his'], ['His father', 'him', "His father's", 'My father, Theodore Roosevelt', 'businessma

In [ ]:
with open('coref_terms.txt', '+a') as coref_file:
    for cluster in coref_terms_clusters:    
        coref_file.write('+'.join(cluster))
        coref_file.write('\n')
    coref_file.close()

Tự chỉnh sửa các term mang tính đại diện nhất của mỗi cụm(trường hợp này mỗi cụm là 1 dòng) nếu cần thiết. Sau đó chèn vào đầu mỗi dòng các term đại diện đấy kèm thêm dấu '+' nối vào đoạn sau.

Tiếp đó là đọc file trên rồi lấy từ đầu tiên trong các dòng(cụm) rồi thay vào trong văn bản gốc.

Sau cùng, join các term đã lọc được sử dụng '_' thay cho ' ' giữa các từ trong term. Từ đó thay các term mới sửa này vào văn bản thay cho term gốc, rồi tokenize văn bản thành list vocab rồi cho vào word2vec để train.

In [ ]:
with open('coref_terms.txt', 'r') as coref_file:
    final_coref_term = []
    a = coref_file.readline()
    while(a != ''):
        final_coref_term.append((a.split('+')[0].split('\n'))[0])
        a = coref_file.readline()
    coref_file.close()

In [ ]:
print(len(final_coref_term))
print(len(FCoref_terms_indextype))

40
40


In [ ]:
for i in range(len(final_coref_term)):
    print(i)
    for index in FCoref_terms_indextype[i]:
        print(index)
        text.replace(text[index[0]+index[1]], final_coref_term[i])

print(text)

0
(0, 77)
(57, 60)
(250, 252)
(472, 481)
(635, 637)
(647, 650)
(670, 672)
(714, 723)
(735, 738)
(863, 865)
(960, 963)
(1010, 1013)
(1092, 1101)
(1190, 1193)
(1252, 1255)
(1273, 1275)
(1343, 1352)
(1492, 1494)
(1630, 1639)
(1727, 1730)
(1781, 1784)
(1827, 1836)
(1972, 1981)
(2130, 2132)
(2144, 2147)
(2296, 2305)
(2449, 2451)
(2486, 2488)
(2529, 2538)
(2637, 2640)
(2708, 2711)
(2747, 2750)
(2797, 2806)
(2941, 2944)
(2957, 2959)
(2968, 2971)
(3014, 3017)
(3053, 3062)
(3177, 3179)
(3239, 3241)
(3383, 3392)
(3447, 3449)
(3503, 3505)
(3621, 3630)
(3683, 3686)
(3723, 3725)
(3790, 3793)
(3871, 3889)
(3900, 3922)
(4005, 4007)
(4139, 4141)
(4297, 4307)
(4696, 4706)
(4945, 4954)
(4996, 5007)
(5036, 5039)
(5077, 5079)
(5212, 5220)
(5225, 5228)
(5273, 5275)
(5321, 5324)
(5378, 5380)
(5449, 5458)


IndexError: string index out of range

In [ ]:
print(text[57:60])

his


In [ ]:
print(len('Theodore Roosevelt Jr., often referred to as Teddy or by his initials, T. R.,'))

77
